In [1]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0,  
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/retail-assistant-athlete-wembley-20-hours-3-month-ftc/job/R-67219")
page_data = loader.load().pop().page_content
print(page_data)





















Retail Assistant (Athlete) - Wembley - 20 Hours - 3 month FTC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language













In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):    
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Retail Assistant (Athlete)",
  "experience": "Customer service and/or retail experience preferred",
  "skills": [
    "Effective communicator",
    "Brand ambassador",
    "Collaborative teammate",
    "Product sales techniques"
  ],
  "description": "Nike Wembley is looking for a Retail Assistant to join our team and provide world-class service to the consumer. This is a 3 month fixed-term contract. The successful candidate will use their expertise in exceptional consumer service and product knowledge to serve and deliver a premium brand experience to our consumers."
}
```


In [6]:
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

jsonparser = JsonOutputParser()
json_res = jsonparser.parse(res.content)
print(json_res)

{'role': 'Retail Assistant (Athlete)', 'experience': 'Customer service and/or retail experience preferred', 'skills': ['Effective communicator', 'Brand ambassador', 'Collaborative teammate', 'Product sales techniques'], 'description': 'Nike Wembley is looking for a Retail Assistant to join our team and provide world-class service to the consumer. This is a 3 month fixed-term contract. The successful candidate will use their expertise in exceptional consumer service and product knowledge to serve and deliver a premium brand experience to our consumers.'}


In [8]:
type(json_res)

dict

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [11]:
links = collection.query(query_texts=["Experience in python","Experience in React"],
                         n_results=2).get('metadatas')

links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [12]:
job = json_res
job['skills']

['Effective communicator',
 'Brand ambassador',
 'Collaborative teammate',
 'Product sales techniques']

In [13]:
job

{'role': 'Retail Assistant (Athlete)',
 'experience': 'Customer service and/or retail experience preferred',
 'skills': ['Effective communicator',
  'Brand ambassador',
  'Collaborative teammate',
  'Product sales techniques'],
 'description': 'Nike Wembley is looking for a Retail Assistant to join our team and provide world-class service to the consumer. This is a 3 month fixed-term contract. The successful candidate will use their expertise in exceptional consumer service and product knowledge to serve and deliver a premium brand experience to our consumers.'}

In [14]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [15]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Harsh, a senior data scientist manager at XYZ. XYZ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase XYZ's portfolio: {link_list}
        Remember you are Harsh, SDS at XYZ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Retail Experience with AI-Driven Solutions for Nike Wembley

Dear Hiring Manager,

I came across the job posting for a Retail Assistant (Athlete) at Nike Wembley and was impressed by the emphasis on providing world-class service to consumers. As a Senior Data Scientist Manager at XYZ, I believe our company can help enhance the retail experience for Nike Wembley through our AI-driven solutions.

At XYZ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in Android app development (https://example.com/android-portfolio) can help create immersive brand experiences, while our capabilities in Magento (https://example.com/magento-portfolio) and WordPress (https://example.com/wordpress-portfolio) can streamline e-commerce operations.

Moreover, our experience in machine learning with Python (https://example.com/ml-python-portfolio) can